# Enzyme with GPU 

Now, we want to take the previous examples and do similar examples on GPU 

* 1) with CUDA.jl 
* 2) with KernelAbstractions

With CUDA, as far as I know, only writing your own kernels is supported, not cuBLAS. All of my attempts to differentiate e.g. a standard `mul!` don't work. 

NOTE: The part with CUDA.jl will only work on GPUs, the part on KernelAbstraction works both on GPU and CPU

## Enzyme with CUDA.jl

We take the last example from the CPU notebook and try to do it on GPU with `CUDA.jl`

For this purpose we have to write CUDA kernels

In [1]:
pwd()

"/Users/max/Nextcloud/SpeedyExperiments/scripts"

In [2]:
import Pkg 
Pkg.activate(".") # make sure this is really the right environment
using Enzyme, Test, CUDA, SpeedyExperiments, LinearAlgebra, Adapt
CUDA.allowscalar(false)

  Activating project at `~/Nextcloud/SpeedyExperiments/scripts`


Should be the scripts folder, otherwise change it. If you are using it for the first time, you might need to `]dev ..` the package again. In the `SpeedyExperiments` there are some utilities for GPU usage, like e.g:

In [3]:
?DeviceArray

search: DeviceArray CuDeviceArray DeviceSparseArray CuDeviceMatrix



```
DeviceArray(x)
```

Returns a `CuArray` when CUDA is used, otherwise a regular `Array`


We adjust the example from the other notebook, but this time we use the GPU. In order to save some time writing the kernel, we just do an elementwise multiplication. Now let's try to take the derivative, for this we use `autodiff_deferred` instead of the regular `autodiff`. The syntax is the same, but it is adjusted for GPU usage. Note that we have to execute this as a kernel as well. 

In [4]:
function element_mul_kernel!(C, A, B)
    i = threadIdx().x
    C[i] = A[i]*B[i]
    return nothing
end

function grad_mul_kernel!(C, dC, A, dA, B, dB)
    Enzyme.autodiff_deferred(element_mul_kernel!, Const, Duplicated(C, dC), Duplicated(A, dA), Duplicated(B, dB))
    return nothing
end

X_1 = DeviceArray(rand(3,3))
∂X_1 = zero(X_1) # input, hence zero 
X_2 = DeviceArray(rand(size(X_1)...))
∂X_2 = zero(X_2) # input, hence zero
Y = DeviceArray(zeros(size(X_1)...))
∂Y = fill!(similar(Y), 1); # output, hence something

In [5]:
@cuda threads=length(X_1) grad_mul_kernel!(Y, ∂Y, X_1, ∂X_1, X_2, ∂X_2)

CUDA.HostKernel{typeof(grad_mul_kernel!), NTuple{6, CuDeviceMatrix{Float64, 1}}}(grad_mul_kernel!, CuFunction(Ptr{Nothing} @0x000000000310c010, CuModule(Ptr{Nothing} @0x0000000005c623d0, CuContext(0x0000000002b402c0, instance e0ff3f39bda82691))), CUDA.KernelState(Ptr{Nothing} @0x00002b20e5a00000))

In [6]:
@test Y ≈ X_1 .* X_2

Test Passed
  Expression: Y ≈ X_1 .* X_2
   Evaluated: [0.518367141735663 0.04514307627053391 0.183222686204985; 0.43110858915650235 0.3155912165251231 0.13808752179289513; 0.2107084030051926 0.06296262389166143 0.1821020051992087] ≈ [0.518367141735663 0.04514307627053391 0.183222686204985; 0.43110858915650235 0.3155912165251231 0.13808752179289513; 0.2107084030051926 0.06296262389166143 0.1821020051992087]

In [7]:
@test ∂X_1 ≈ X_2

Test Passed
  Expression: ∂X_1 ≈ X_2
   Evaluated: [0.5825494050811111 0.9523803367679741 0.3894609012488429; 0.5902793013826094 0.5402133039887466 0.8606560948422022; 0.23473898524613512 0.08905694353173121 0.7137234003704515] ≈ [0.5825494050811111 0.9523803367679741 0.3894609012488429; 0.5902793013826094 0.5402133039887466 0.8606560948422022; 0.23473898524613512 0.08905694353173121 0.7137234003704515]

In [8]:
@test ∂X_2 ≈ X_1

Test Passed
  Expression: ∂X_2 ≈ X_1
   Evaluated: [0.8898252014582151 0.04740026072328707 0.4704520675052727; 0.7303467835424993 0.5841974164555143 0.1604444825528284; 0.8976284990933854 0.7069928676502096 0.25514366644653985] ≈ [0.8898252014582151 0.04740026072328707 0.4704520675052727; 0.7303467835424993 0.5841974164555143 0.1604444825528284; 0.8976284990933854 0.7069928676502096 0.25514366644653985]

Now, we do the same but with a struct.

 If we use CUDA with custom structs, we have to make sure to use `Adapt` to make our structs avaliable to CUDA, [as explained here](https://cuda.juliagpu.org/stable/tutorials/custom_structs/). The struct also has to have (parametric) types defined, i.e without the `{S,T,U}` we would get an error!

In [9]:
struct PreComputeMul{S,T,U} 
    Y::S
    X_1::T
    X_2::U
    
    function PreComputeMul(X,Y,Z) 
        @assert length(X) == length(Y) == length(Z)
        new{typeof(X),typeof(Y),typeof(Z)}(X,Y,Z)
    end 
end

function Adapt.adapt_structure(to, m::PreComputeMul)
    Y = Adapt.adapt_structure(to, m.Y)
    X_1 = Adapt.adapt_structure(to, m.X_1)
    X_2 = Adapt.adapt_structure(to, m.X_2)
    PreComputeMul(Y, X_1, X_2)
end

function element_mul_kernel!(C::PreComputeMul)
    i = threadIdx().x
    C.Y[i] = C.X_1[i]*C.X_2[i]
    return nothing
end

function compute_elementwise_mul!(X::PreComputeMul)
    @cuda threads=length(X.Y) element_mul_kernel!(X.Y, X.X_1, X.X_2)
end 

compute_elementwise_mul! (generic function with 1 method)

In [10]:
X_1 = DeviceArray(rand(3,3))
∂X_1 = zero(X_1) # input, hence zero 
X_2 = DeviceArray(rand(size(X_1)...))
∂X_2 = zero(X_2) # input, hence zero
Y = DeviceArray(zeros(size(X_1)...))
∂Y = fill!(similar(Y), 1) # output, hence something

X = PreComputeMul(Y, X_1, X_2)
∂X = PreComputeMul(∂Y, ∂X_1, ∂X_2)

PreComputeMul{CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}, CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}, CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}}([1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0], [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0], [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0])

The first way to compute this would also work without the `adapt_structure`:

In [11]:
@cuda threads=length(X.Y) element_mul_kernel!(X.Y, X.X_1, X.X_2)

CUDA.HostKernel{typeof(element_mul_kernel!), Tuple{CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}}}(element_mul_kernel!, CuFunction(Ptr{Nothing} @0x0000000011905e60, CuModule(Ptr{Nothing} @0x0000000011905cb0, CuContext(0x0000000002b402c0, instance e0ff3f39bda82691))), CUDA.KernelState(Ptr{Nothing} @0x00002b20e5a00000))

But for kernels that actually use the structs, we need it:

In [12]:
@cuda threads=length(X.Y) element_mul_kernel!(X)

CUDA.HostKernel{typeof(element_mul_kernel!), Tuple{PreComputeMul{CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}}}}(element_mul_kernel!, CuFunction(Ptr{Nothing} @0x00000000115165b0, CuModule(Ptr{Nothing} @0x0000000007fb2fb0, CuContext(0x0000000002b402c0, instance e0ff3f39bda82691))), CUDA.KernelState(Ptr{Nothing} @0x00002b20e5a00000))

Test if the implementation works on GPU

In [13]:
@test X.Y ≈ X.X_1 .* X.X_2

Test Passed
  Expression: X.Y ≈ X.X_1 .* X.X_2
   Evaluated: [0.12712573852891057 0.23446725910034724 0.012028230568762222; 0.16900630393232668 0.6076334912688396 0.26773287122182476; 0.04430743480845625 0.06708683278909719 0.41290929450218006] ≈ [0.12712573852891057 0.23446725910034724 0.012028230568762222; 0.16900630393232668 0.6076334912688396 0.26773287122182476; 0.04430743480845625 0.06708683278909719 0.41290929450218006]

Now, we can take the derivative similar to the example without the struct

In [14]:
function grad_elementwise_mul_kernel!(A, dA)
    Enzyme.autodiff_deferred(element_mul_kernel!, Const, Duplicated(A, dA))
    return nothing
end

grad_elementwise_mul_kernel! (generic function with 1 method)

In [15]:
@cuda threads=length(X.Y) grad_elementwise_mul_kernel!(X, ∂X)

CUDA.HostKernel{typeof(grad_elementwise_mul_kernel!), Tuple{PreComputeMul{CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}}, PreComputeMul{CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}, CuDeviceMatrix{Float64, 1}}}}(grad_elementwise_mul_kernel!, CuFunction(Ptr{Nothing} @0x00000000121f7a10, CuModule(Ptr{Nothing} @0x000000001163bac0, CuContext(0x0000000002b402c0, instance e0ff3f39bda82691))), CUDA.KernelState(Ptr{Nothing} @0x00002b20e5a00000))

In [16]:
@test ∂X_1 ≈ X_2

Test Passed
  Expression: ∂X_1 ≈ X_2
   Evaluated: [0.6757225470713533 0.6629617469870471 0.48239405723117035; 0.3404947156585928 0.708240345050117 0.28412692076096935; 0.1254104958726736 0.07725581087465527 0.7143499334334222] ≈ [0.6757225470713533 0.6629617469870471 0.48239405723117035; 0.3404947156585928 0.708240345050117 0.28412692076096935; 0.1254104958726736 0.07725581087465527 0.7143499334334222]

In [17]:
@test ∂X_2 ≈ X_1

Test Passed
  Expression: ∂X_2 ≈ X_1
   Evaluated: [0.1881330422965547 0.35366634676273145 0.02493445014186424; 0.4963551449115172 0.8579481464386808 0.9423002597035267; 0.353299255378438 0.8683726444596267 0.5780210442767031] ≈ [0.1881330422965547 0.35366634676273145 0.02493445014186424; 0.4963551449115172 0.8579481464386808 0.9423002597035267; 0.353299255378438 0.8683726444596267 0.5780210442767031]

## Enzyme with KernelAbstractions.jl 

Now we do the same but with `KernelAbstractions`. `KernelAbstractions` has the advantage that it works both on CPU and GPU, as we will also see in these examples. 

In [3]:
using KernelAbstractions, CUDAKernels, KernelGradients 

The syntax for the kernel is almost the same! The `@Const` marks input arguments that are not allowed to be mutated or aliases of other input arguments. It would work without it, but this optimizes the kernel. 


In [4]:
@kernel function KA_element_mul_kernel!(C, @Const(A), @Const(B))
    i, j = @index(Global, NTuple)
    C[i,j] = A[i,j] * B[i,j]
end

KA_element_mul_kernel! (generic function with 5 methods)

We have to launch it a bit different though. First we define a wrapper around the kernel

In [5]:
function element_mul!(a, b, c)
    @assert size(a) == size(b) == size(c)
  
    device = KernelAbstractions.get_device(a) # here we determine if the array is on GPU or CPU
    n = device isa GPU ? 256 : 4   # we split how 
    kernel! = KA_element_mul_kernel!(device, n)
    kernel!(a, b, c, ndrange=size(c)) 
end


element_mul! (generic function with 1 method)

Then, we launch it. With KernelAbstractions, we have to wait for all computations to be done manually.

In [6]:
X_1 = DeviceArray(rand(256,256))
X_2 = DeviceArray(rand(size(X_1)...))
Y = DeviceArray(zeros(size(X_1)...))

ev = element_mul!(Y, X_1, X_2)
wait(ev)

In [7]:
@test Y ≈ X_1 .* X_2

Test Passed
  Expression: Y ≈ X_1 .* X_2
   Evaluated: [0.004080368546444639 0.9043155088865072 … 0.07077905252442658 0.05363192270551029; 0.16882664261346356 0.5866150330869437 … 0.08470709940218783 0.026242189387002193; … ; 0.041947544993145985 0.6439503497476583 … 0.22806647617207776 0.0064231973111595715; 0.10623230790522847 0.027091019924623583 … 0.05309581572181774 0.12675056890341813] ≈ [0.004080368546444639 0.9043155088865072 … 0.07077905252442658 0.05363192270551029; 0.16882664261346356 0.5866150330869437 … 0.08470709940218783 0.026242189387002193; … ; 0.041947544993145985 0.6439503497476583 … 0.22806647617207776 0.0064231973111595715; 0.10623230790522847 0.027091019924623583 … 0.05309581572181774 0.12675056890341813]

If we'd want to, we could also write this a bit more compact: 

In [8]:
?SpeedyExperiments.Device

```
Device()
```

Return currently used device for KernelAbstractions, either `CPU` or `CUDADevice`


In [9]:
X_1 = DeviceArray(rand(256,256))
X_2 = DeviceArray(rand(size(X_1)...))
Y = DeviceArray(zeros(size(X_1)...))

const device = SpeedyExperiments.Device()
n = device isa GPU ? 256 : 4   # we split how 

wait(KA_element_mul_kernel!(device(), n)(Y, X_1, X_2, ndrange=size(Y)))

In [10]:
@test Y ≈ X_1 .* X_2

Test Passed
  Expression: Y ≈ X_1 .* X_2
   Evaluated: [0.031525836235158214 0.07597293627416002 … 0.3900124829967884 0.24506803919395512; 0.02803691703362597 0.4145321860192378 … 0.25451746863793495 0.18423061770888527; … ; 0.07389615398436922 0.0615598767008774 … 0.6303968501185453 0.6018271499495584; 0.32663465913853057 0.04012554055577644 … 0.5291158689856843 0.4236591763498572] ≈ [0.031525836235158214 0.07597293627416002 … 0.3900124829967884 0.24506803919395512; 0.02803691703362597 0.4145321860192378 … 0.25451746863793495 0.18423061770888527; … ; 0.07389615398436922 0.0615598767008774 … 0.6303968501185453 0.6018271499495584; 0.32663465913853057 0.04012554055577644 … 0.5291158689856843 0.4236591763498572]

Now, let's compute the derivatives for this we have to call `autodiff` with the kernel function which in turn is called with the device and the work group size. Then we can call the kernel event created by `autodiff` with the `Duplicated` inputs 

In [11]:
X_1 = DeviceArray(rand(256,256))
∂X_1 = zero(X_1) # input, hence zero 
X_2 = DeviceArray(rand(size(X_1)...))
∂X_2 = zero(X_2) # input, hence zero
Y = DeviceArray(zeros(size(X_1)...))
∂Y = fill!(similar(Y), 1); # output, hence something

In [12]:
∇! = autodiff(KA_element_mul_kernel!(device(), n))
wait(∇!(Duplicated(Y, ∂Y), Duplicated(X_1,∂X_1), Duplicated(X_2,∂X_2); ndrange=size(Y)))

In [13]:
@test ∂X_2 ≈ X_1

Test Passed
  Expression: ∂X_2 ≈ X_1
   Evaluated: [0.4849006986370975 0.37813378438950895 … 0.8475662728169604 0.430769784826331; 0.9230358561586566 0.3960121700759315 … 0.44342469674852125 0.15950409722012893; … ; 0.6832069236075147 0.44790489127020927 … 0.8043176706509493 0.28177073185566814; 0.6623945231047209 0.11891028444588247 … 0.3972615116578413 0.6856090205211397] ≈ [0.4849006986370975 0.37813378438950895 … 0.8475662728169604 0.430769784826331; 0.9230358561586566 0.3960121700759315 … 0.44342469674852125 0.15950409722012893; … ; 0.6832069236075147 0.44790489127020927 … 0.8043176706509493 0.28177073185566814; 0.6623945231047209 0.11891028444588247 … 0.3972615116578413 0.6856090205211397]

In [14]:
@test ∂X_1 ≈ X_2

Test Passed
  Expression: ∂X_1 ≈ X_2
   Evaluated: [0.8592885121353061 0.4278155350603593 … 0.15727044681588565 0.7475614986843266; 0.8717433078125162 0.4941278872225551 … 0.44855984012001493 0.9911128063246805; … ; 0.9182669423034203 0.11752918046760064 … 0.925919239995262 0.36151376995133777; 0.7928602512054792 0.5957790896537778 … 0.5514845686998056 0.4636690929932934] ≈ [0.8592885121353061 0.4278155350603593 … 0.15727044681588565 0.7475614986843266; 0.8717433078125162 0.4941278872225551 … 0.44855984012001493 0.9911128063246805; … ; 0.9182669423034203 0.11752918046760064 … 0.925919239995262 0.36151376995133777; 0.7928602512054792 0.5957790896537778 … 0.5514845686998056 0.4636690929932934]

Now, let's do this with a struct. First just the regular forward execution:

In [15]:
struct KAPreComputeMul{S,T,U} 
    Y::S
    X_1::T
    X_2::U
    
    function KAPreComputeMul(X,Y,Z) 
        @assert size(X) == size(Y) == size(Z)
        new{typeof(X),typeof(Y),typeof(Z)}(X,Y,Z)
    end 
end

@kernel function KAstruct_element_mul_kernel!(A)
    i, j = @index(Global, NTuple)
    A.Y[i,j] = A.X_1[i,j] * A.X_2[i,j]
end




KAstruct_element_mul_kernel! (generic function with 5 methods)

In [17]:
X_1 = DeviceArray(rand(256,256))
X_2 = DeviceArray(rand(size(X_1)...))
Y = DeviceArray(zeros(size(X_1)...))

X = KAPreComputeMul(Y, X_1, X_2);

In [18]:
wait(KAstruct_element_mul_kernel!(device(), n)(X, ndrange=size(X.Y)))

In [19]:
@test Y ≈ X_1 .* X_2

Test Passed
  Expression: Y ≈ X_1 .* X_2
   Evaluated: [0.3301541209620804 0.580555896689647 … 0.17765169008007817 0.03290889765257146; 0.20139468466718466 0.25206788278054865 … 0.16659489071421682 0.0451513064248377; … ; 0.318866316385818 0.1373982105414804 … 0.48359294501357847 0.7998793193429918; 0.05451786957137616 0.07832617240755971 … 0.5352945461050356 0.13283968682258013] ≈ [0.3301541209620804 0.580555896689647 … 0.17765169008007817 0.03290889765257146; 0.20139468466718466 0.25206788278054865 … 0.16659489071421682 0.0451513064248377; … ; 0.318866316385818 0.1373982105414804 … 0.48359294501357847 0.7998793193429918; 0.05451786957137616 0.07832617240755971 … 0.5352945461050356 0.13283968682258013]

Great, now let's also compute some derivatives here:

In [21]:
X_1 = DeviceArray(rand(256,256))
∂X_1 = zero(X_1) # input, hence zero 
X_2 = DeviceArray(rand(size(X_1)...))
∂X_2 = zero(X_2) # input, hence zero
Y = DeviceArray(zeros(size(X_1)...))
∂Y = fill!(similar(Y), 1) # output, hence something

X = KAPreComputeMul(Y, X_1, X_2)
∂X = KAPreComputeMul(∂Y, ∂X_1, ∂X_2)

KAPreComputeMul{Matrix{Float64}, Matrix{Float64}, Matrix{Float64}}([1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [22]:
∇! = autodiff(KAstruct_element_mul_kernel!(device(), n))
wait(∇!(Duplicated(X, ∂X); ndrange=size(Y)))

In [23]:
@test ∂X_2 ≈ X_1

Test Passed
  Expression: ∂X_2 ≈ X_1
   Evaluated: [0.29979379103443005 0.9235020762350673 … 0.2868482883717578 0.25899853314330157; 0.10217358278136734 0.5676732372408608 … 0.24207432814467766 0.8005602687345921; … ; 0.14158078085515546 0.17066995013824293 … 0.731331194896376 0.7818542661462423; 0.858094764413582 0.8342561525161541 … 0.8052198677497796 0.011702558521468065] ≈ [0.29979379103443005 0.9235020762350673 … 0.2868482883717578 0.25899853314330157; 0.10217358278136734 0.5676732372408608 … 0.24207432814467766 0.8005602687345921; … ; 0.14158078085515546 0.17066995013824293 … 0.731331194896376 0.7818542661462423; 0.858094764413582 0.8342561525161541 … 0.8052198677497796 0.011702558521468065]

In [24]:
@test ∂X_1 ≈ X_2

Test Passed
  Expression: ∂X_1 ≈ X_2
   Evaluated: [0.28407138003919263 0.583496276232759 … 0.8142553122082264 0.2764854349908631; 0.2078496455773844 0.8886070526803997 … 0.43291726250025897 0.9802051008855949; … ; 0.7974184596683417 0.7770545468535559 … 0.7125297874928762 0.46376083130345935; 0.17201805825290362 0.8998755117729266 … 0.36138431358805123 0.733282888241287] ≈ [0.28407138003919263 0.583496276232759 … 0.8142553122082264 0.2764854349908631; 0.2078496455773844 0.8886070526803997 … 0.43291726250025897 0.9802051008855949; … ; 0.7974184596683417 0.7770545468535559 … 0.7125297874928762 0.46376083130345935; 0.17201805825290362 0.8998755117729266 … 0.36138431358805123 0.733282888241287]